# Assignment 2 - Classification - KNN

Dataset - The Titanic dataset: <br/>
<img src="ships-titanic-vehicles-best.jpg" Width="800"/>

In [53]:
# import file section - DO NOT CHANGE:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from random import randrange

## General instructions:
Please Read the following instructions, and make sure you follow all steps, before submiting your assignment via moodle:
- Put your code one line after the '# YOUR CODE HERE' remark
- You must remove the 'raise NotImplementedError()' exception raising line (which appears a line after the above remark).
    * if you do not remove this line, it will be a sign that you didn't implement that code, and we won't be able to check your work
- Do NOT remove any other line in this notebook    
- you also need to implement the two functions 'myName', 'myId'
    * myName - you need to return your full name as a string
    * myId - you need to return your ID number as a an integer    
- When you want to check your work, select the 'Cell' --> 'Run All' menu
    * before performing your filnal test, we suggest to clear previous output (by selecting 'Cell' --> 'All output' --> 'Clear' menu) and then reperform the final execution ('Cell' --> 'Run All') of the code.
    * after performing 'Run All', make sure there are no exceptions thrown and that the output is as you expected.
- Don't forget to save your work, by clicking the 'save' icon (in the upper left part of the screen), or by selecting the 'File' --> 'Save and checkpoint' menu item. 
- Do NOT change the file name of this notebook
- The work is done individualy, for each student
- Each student needs to submit his/her assignment through moodle.
- Submit the python notebook only (not any additional possible files) 
<br/><br/>
Good Luck :-)<br/>
The courses staff

## The following 2 functions are mandatory to get your details

In [3]:
# Change the following function to return your full name (as a string).
# --------------------- General notes:
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def myName():
    return "Menahem Ovrutski"

In [4]:
# Change the following function to return your ID number (as an integer number).
# --------------------- General notes:
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def myId():
    return 123456789

In [5]:
aName = myName()
aId = myId()
assert aName is not None, 'no return student name'
assert type(aName) is str, "name is not a string: %r" % aName
assert aId is not None, 'no return student id'
assert type(aId) is int, "id is not an integer: %r" % aId
print ("your name is: %s" %(aName))
print ("your id is: %d" %(aId))

your name is: Menahem Ovrutski
your id is: 123456789


In [57]:
# Change the following function to return two datasets xInstances, yCategories
# The xInstances contains all columns, besides the 'Survived' column, which is the category column
# ----- the xInstances is a dataframe, with the 'PassengerId' column as an index.
# ----- the xInstances dataframe contains the instance vectors (each row is an instance with values of the features) 
# The yCategories contains only two columns: 'PassengerId','Survived' (which is a column of the categories) 
# ----- the yCategories is a dataframe, with the 'PassengerId' column as an index.
# ----- the yCategories dataframe contains only the category column ('Survived'), besides the index 
# ----- (each value in the 'Survived' column corresponds to the feature vector in xInstances with the same 'PassengerId') 
# --------------------- General notes:
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def loadCsvFile(fileName):
    dfMain = pd.read_csv(fileName)
    xInstances = dfMain.drop('Survived',axis=1)
    xInstances = xInstances.set_index('PassengerId')
    yInstances = dfMain[['PassengerId','Survived']]
    yInstances = yInstances.set_index('PassengerId')
    return xInstances,yInstances

In [58]:
xTitanicInstances, yTitanicCategories = loadCsvFile('titanic_dataset_fix.csv')
# validating that the data is loaded successfully:
assert xTitanicInstances is not None, 'titanic instance dataframe object is empty'
print ( 'titanic instance dataframe object is NOT empty')
assert 'Survived' not in xTitanicInstances.columns, 'instances should not include the category column'
assert np.array_equal(xTitanicInstances['Age'].head().tolist()[1:3],[38, 26]), 'titanic instance dataframe object was NOT loaded successfully'
print ("\t titanic instance dataframe object was loaded successfuly")
assert yTitanicCategories is not None, 'titanic category dataframe object is empty'
assert 'Survived' in yTitanicCategories.columns, 'category dataframe is missing the category column'
print ( 'titanic category dataframe object is NOT empty')
assert np.array_equal(yTitanicCategories['Survived'].head().tolist()[:2],[0, 1]), 'titanic category dataframe object was NOT loaded successfully'
print ("\t titanic category dataframe object was loaded successfuly")
print ('------------------------\n')

  
print ('instance (=feature vector) datafame (displaying the "age","gender" features):')
print (xTitanicInstances[['Age','Gender']].head())
print ('------------------------\n')
print ('class (=category) datafame:')
print (yTitanicCategories.head())
print ('------------------------\n')


titanic instance dataframe object is NOT empty
	 titanic instance dataframe object was loaded successfuly
titanic category dataframe object is NOT empty
	 titanic category dataframe object was loaded successfuly
------------------------

instance (=feature vector) datafame (displaying the "age","gender" features):
              Age  Gender
PassengerId              
1            22.0    male
2            38.0  female
3            26.0  female
4            35.0  female
5            35.0    male
------------------------

class (=category) datafame:
             Survived
PassengerId          
1                   0
2                   1
3                   1
4                   1
5                   0
------------------------



In [59]:
# Change the following function to return a train-test split of the dataset
# --- hint: use the randrange function, taught in class
# input parameter: xInstances - the instance dataframe (=feature vectors) without the categories  
# input parameter: yCategories - the category (=class/tag) dataframe
# input parameter: testRatio - a 0<number<1, which represent the ratio of the dataset which will be used by the test set
# --- as explained, frequently used values are: 0.1, 0.2 or 0.3
# output: xTrainInstances, xTestInstances, yTrainCategories, yTestCategories
# --------------------- General notes:
# Use of external packages are forbidden
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def trainTestSplit(xInstances,yCategories,testRatio):
    # YOUR CODE HERE
    xTestSet = set()
    yTestSet = set()
    xInstances_Size = len(xInstances)
    yCategories_Size = len(yCategories)
    while(len(xTestSet) < testRatio * xInstances_Size):
        xTestSet.add(randrange(1,xInstances_Size))
    while(len(yTestSet) < testRatio * yCategories_Size):
        yTestSet.add(randrange(1,yCategories_Size))
    xInstancesColumns = xInstances.columns
    yCategoriesColumns = yCategories.columns
    xTrain = pd.DataFrame(columns=xInstancesColumns)
    xTest =  pd.DataFrame(columns=xInstancesColumns)
    yTrain = pd.DataFrame(columns=yCategoriesColumns)
    yTest =  pd.DataFrame(columns=yCategoriesColumns)
    for x in range(1,len(xInstances)+1):
        if x in xTestSet:
            xTest.loc[x] = xInstances.loc[x]
        else:
            xTrain.loc[x] = xInstances.loc[x]
    for y in range(1,len(yCategories) + 1):
        if y in yTestSet:
            yTest.loc[y] = yCategories.loc[y]
        else:
            yTrain.loc[y] = yCategories.loc[y]
    return xTrain,xTest,yTrain,yTest
    
    #raise NotImplementedError()

In [60]:
xTitanicInstances10 = xTitanicInstances.iloc[:10]
yTitanicCategories10= yTitanicCategories.iloc[:10]

from random import seed
seed(1)
splitSize=0.3
xTrain,xTest,yTrain,yTest = trainTestSplit(xTitanicInstances10,yTitanicCategories10,splitSize)
assert xTest is not None, 'xTest should not be None'
assert len(xTest)==len(xTitanicInstances10)*splitSize,'wrong split size'
assert yTest is not None, 'yTest should not be None'
assert len(yTest)==len(xTitanicInstances10)*splitSize,'wrong split size'
assert xTrain is not None, 'xTrain should not be None'
assert len(xTrain)==len(xTitanicInstances10)*(1-splitSize),'wrong split size'
assert yTrain is not None, 'yTrain should not be None'
assert len(yTrain)==len(xTitanicInstances10)*(1-splitSize),'wrong split size'

print(xTrain[['Age','Gender']])
print('-----------------')
print(xTest[['Age','Gender']])
print('-----------------')



          Age  Gender
1   22.000000    male
4   35.000000  female
6   41.560785    male
7   54.000000    male
8    2.000000    male
9   27.000000  female
10  14.000000  female
-----------------
    Age  Gender
2  38.0  female
3  26.0  female
5  35.0    male
-----------------


In [61]:
# Change the following function to return the Euclidean Distastance between two vectors from the test and train
# xSeriesTestVector and xSeriesTrainVector are actually Series objects
# ---- the xSeriesTestVector and xSeriesTrainVector consist of a single row (in the from xTrain and xTest correspondigly) 
# The xInstances contains all columns, besides the 'Survived' column, which is the category column
# --------------------- General notes:
# Use of external packages are forbidden
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def euclideanDist(xSeriesTestVector, xSeriesTrainVector):
    # YOUR CODE HERE
    dist = 0
    for i in range(len(xSeriesTestVector)):
        dist+=(xSeriesTestVector[i] - xSeriesTrainVector[i])**2 
    return dist**0.5
    #raise NotImplementedError()


In [62]:
inst0 = xTitanicInstances[['Pclass','SibSp','Parch']].iloc[0]
inst7 = xTitanicInstances[['Pclass','SibSp','Parch']].iloc[7]
print (inst0.values)
print (inst7.values)
dist = euclideanDist(inst0,inst7)
assert float(int(dist*100))/100 == 2.23, 'unexpected euclidean distance'
print ('Euclidean Distance calculated successfuly: %f' %(dist))

[3 1 0]
[3 3 1]
Euclidean Distance calculated successfuly: 2.236068


In [63]:
# Change the following function to return the Manhattan Distastance between two vectors from the test and train
# xSeriesTestVector and xSeriesTrainVector are actually Series objects
# ---- the xSeriesTestVector and xSeriesTrainVector consist of a single row (in the from xTrain and xTest correspondigly) 
# The xInstances contains all columns, besides the 'Survived' column, which is the category column
# --------------------- General notes:
# Use of external packages are forbidden
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# --------------------- 
def manhattanDist(xSeriesTestVector,xSeriesTrainVector):
    # YOUR CODE HERE
    ManDist = 0
    value = 0
    for i in range(len(xSeriesTestVector)):
        value = (xSeriesTestVector[i] - xSeriesTrainVector[i])
        if value < 0:
            value *= (-1)
        ManDist += value
    return ManDist
    #raise NotImplementedError()


In [64]:
inst0 = xTitanicInstances[['Pclass','SibSp','Parch']].iloc[0]
inst7 = xTitanicInstances[['Pclass','SibSp','Parch']].iloc[7]
print (inst0.values)
print (inst7.values)
dist = manhattanDist(inst0,inst7)
assert dist== 3, 'unexpected manhattan distance'
print ('Manhattan Distance calculated successfuly: %d' %(dist))

[3 1 0]
[3 3 1]
Manhattan Distance calculated successfuly: 3


In [65]:
# Change the following function to return the Distastance between a test instance and all train instances
# input parameter - xSeriesTestVector - is a Series object, from the test set
# input parameter - xTrainInstances - is a dataframe including all train instances
# input parameter - distanceMetric - the name of the distance function, not as a string
# ---- distanceMetric - options are: euclideanDist ; manhattanDist
# The retuned value is a numpy array, containing distances between the test instance
#                 and all train instances (ordered by the train instance order) 
# --------------------- General notes:
# Use of external packages are forbidden
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# ---------------------
def calcDistances(xSeriesTestVector, xTrainInstances, distanceMetric):
    # YOUR CODE HERE
    size = len(xTrainInstances)
    distances1 = np.zeros(size,np.float32)
    for i in range(size):
        distances1[i] = distanceMetric(xSeriesTestVector, xTrainInstances.iloc[i])
    return distances1
    #raise NotImplementedError()

# Change the following function to predict the expected category, using the k-nn algorithm
# input parameter - xSeriesTestVector - is a Series object, from the test set
# input parameter - xTrainInstances - a dataframe including all train instances
# input parameter - yTrainCategories - a dataframe including all train categories 
# input parameter - distanceMetric - the name of the distance function, not as a string
# ---- distanceMetric - options are: euclideanDist ; manhattanDist
# input parameter - k - the number of Nearest Neighbors (we select the majority out of k votes)
# The retuned value is a numpy array, containing distances between the test instance
#                 and all train instances (ordered by the train instance order) 
# --------------------- General notes:
# Use of external packages are forbidden
# Don't forget to remove the 'raise NotImplementedError()' line.
# In the line following the '# YOUR CODE HERE' line, add your code
# ---------------------
def predict(xSeriesTestVector, xTrainInstances,yTrainCategories,distanceMetric,k):
    distances = calcDistances(xSeriesTestVector, xTrainInstances,distanceMetric)
    # calculate the k- nearest neighbors, based on the above distance array
    # YOUR CODE HERE
    indexD = {}
    for j in range(k):
        minElement = distances[0]
        index = 0
        for i in range(len(distances)):
            if distances[i] < minElement and i not in indexD:
                minElement = distances[i]
                index = i
        indexD[index] = index + 1
    predict = 0
    for key in indexD:
        predict += yTrainCategories['Survived'][indexD[key]]
    if predict <= k / 2:
        return 0
    return 1
    #raise NotImplementedError()


In [66]:
# Feature Scaling
## We will be using standardscaler to transform
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xTitanicInstances_5Features = xTitanicInstances[['Pclass','SibSp','Parch','Fare','Age']]
xTitanicInstances_5Features_40 = xTitanicInstances_5Features.iloc[:40]
yTitanicCategories_40 = yTitanicCategories.iloc[:40]
xTitanicInstances_5Features_TestInst = xTitanicInstances_5Features.iloc[-7:-6]
## transforming "train_x"
xTrainScaled = pd.DataFrame(sc.fit_transform(xTitanicInstances_5Features_40),index=xTitanicInstances_5Features_40.index)

## transforming "test_x"
xTestScaled = pd.Series(sc.transform(xTitanicInstances_5Features_TestInst)[0],index=xTitanicInstances_5Features_40.columns)

# measure distance without scaling:
manhattanDist_k3 = predict(xTitanicInstances_5Features_TestInst.iloc[-1],xTitanicInstances_5Features_40,yTitanicCategories_40,manhattanDist,3)
euclideanDist_k3 = predict(xTitanicInstances_5Features_TestInst.iloc[-1],xTitanicInstances_5Features_40,yTitanicCategories_40,euclideanDist,3)
assert manhattanDist_k3 == 0,"wrong value for knn, with k=3, dist='manhattan'"
assert euclideanDist_k3 == 1,"wrong value for knn, with k=3, dist='euclidean'"
print ('manhattanDist,k=3, inst=-%d, chosen cat: %d' %(7,manhattanDist_k3))
print ('euclideanDist,k=3, inst=-%d, chosen cat: %d' %(7,euclideanDist_k3))
manhattanDist_scaled_k3 = predict(xTestScaled,xTrainScaled,yTitanicCategories_40,manhattanDist,3)
euclideanDist_scaled_k3 = predict(xTestScaled,xTrainScaled,yTitanicCategories_40,euclideanDist,3)
# measure distance with scaling:
assert manhattanDist_scaled_k3 == 1,"wrong value for (scaled) knn, with k=3, dist='manhattan'"
assert euclideanDist_scaled_k3 == 1,"wrong value for (scaled) knn, with k=3, dist='euclidean'"
print ('(scaled) manhattanDist,k=3, inst=-%d, chosen cat: %d' %(7,euclideanDist_scaled_k3))
print ('(scaled) euclideanDist,k=3, inst=-%d, chosen cat: %d' %(7,manhattanDist_scaled_k3))
print ('---------------------------------------------------')



manhattanDist,k=3, inst=-7, chosen cat: 0
euclideanDist,k=3, inst=-7, chosen cat: 1
(scaled) manhattanDist,k=3, inst=-7, chosen cat: 1
(scaled) euclideanDist,k=3, inst=-7, chosen cat: 1
---------------------------------------------------


In [67]:
def evaluate_accuracy(xTestInstances,xTrainInstances,yTrainCategories,yTestCategories,distanceMetric,k):
    # YOUR CODE HERE
    prediction_dict = {}
    i = 1
    test_Series_Size = len(xTestInstances)
    for j in xTestInstances.values:
        prediction_dict[i] = predict(j, xTrainInstances,yTrainCategories,distanceMetric,k)
        i+=1
    mistake = 0
    correct = 0
    i = 1
    for j in yTestCategories.values:
        if prediction_dict[i] == j:
            correct+=1
        else:
            mistake+=1
        i+=1
    return correct/len(yTestCategories)
    # raise NotImplementedError()"

In [68]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

xTitanicInstances_5Features = xTitanicInstances[['Pclass','SibSp','Parch','Fare','Age']]
xTitanicInstances_5Features_100 = xTitanicInstances_5Features.iloc[:100]
yTitanicCategories_100 = yTitanicCategories.iloc[:100]
xTitanicInstances_5Features_Test_10 = xTitanicInstances_5Features.iloc[-20:]
yTitanicCategories_Test_10 = yTitanicCategories.iloc[-20:]
## scale:
xTrainScaled = pd.DataFrame(sc.fit_transform(xTitanicInstances_5Features_100),index=xTitanicInstances_5Features_100.index)
xTestScaled = pd.DataFrame(sc.transform(xTitanicInstances_5Features_Test_10),index=xTitanicInstances_5Features_Test_10.index)

acc_manhattan_3 = evaluate_accuracy(xTitanicInstances_5Features_Test_10,xTitanicInstances_5Features_100,yTitanicCategories_100,yTitanicCategories_Test_10,manhattanDist,3)
acc_scaled_euclidean_3 = evaluate_accuracy(xTestScaled,xTrainScaled,yTitanicCategories_100,yTitanicCategories_Test_10,euclideanDist,3)

assert acc_manhattan_3 == 0.45,"wrong value for accuracy of knn, with k=3, dist='manhattan'"
assert acc_scaled_euclidean_3 == 0.60,"wrong value for accuracy of (Scaled) knn, with k=3, dist='euclidean'"
print ('manhattanDist,k=3, acc: %f' %(acc_manhattan_3))
print ('(Scaled) euclideanDist,k=3, acc: %f' %(acc_scaled_euclidean_3))



manhattanDist,k=3, acc: 0.450000
(Scaled) euclideanDist,k=3, acc: 0.600000
